In [ ]:
import luigi
import pandas as pd
import os
import logging



In [ ]:
class ReadAllData(luigi.Task):
    def run(self):
        df_delinquance = pd.read_csv("./CleanData/delinquance.csv", sep=",")
        df_canton = pd.read_csv("./CleanData/canton.csv", sep=",")
        df_chomeurs = pd.read_csv("./CleanData/chomeurs.csv", sep=",")

        
        merge_data = pd.merge(df_canton, df_delinquance, on='Code du departement', how='inner')
        merge_data = pd.merge(merge_data, df_chomeurs, on='Code du departement', how='inner')

        merge_data = merge_data.drop(["Valeur_x", "Valeur_y", "N°Panneau", 
                                      "Nom", "Prenom", "Sexe", "Libelle du canton",
                                      "Parti polititque",
                                      "Code Tendance",
                                      "Voix", "% Voix/Ins"], axis=1)
        merge_data = merge_data.rename(columns={"faits" : "Faits de délinquance"})

        merge_data['% Voix/Exp'] = merge_data['% Voix/Exp'].replace(',', '.', regex=True).astype(float)
        merge_data = merge_data.groupby(['Code du departement', 'Code du canton', 'Tendance']).agg(lambda x: x.iloc[0] if x.name != '% Voix/Exp' else x.sum()).reset_index()
        merge_data = merge_data.replace(',', '.', regex=True)

        merge_data.to_csv(self.output().path, index=False)

    def output(self):
        return luigi.LocalTarget('./CleanData/data.csv')

In [ ]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)